In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv
import os
import random
import string
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from datetime import datetime
from dateutil.parser import parse
from sklearn import metrics
from unidecode import unidecode
from sklearn.metrics import roc_auc_score

from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [2]:
data_version_input = "01-10-2020"

In [3]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [4]:

f =  "maCv"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("#####################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print(len(a))
print(a)


['-', '- Nhân viên thu phí, soát vé trạm thu phí Km943+975 Công ty TNHH MTV Ceco 545 BOT thuộc Nhóm V.1 - B', '01019882', '22568', '29157- NV Sales DSMs - Vùng II - Nha Trang', '29496-VBL, Nhân viên SP', '309092359 - Công nhân', '39465/NV Bán hàng/Lenovo/vùng 02/Bạc Liêu', '48373/Nhân viên bán hàng LG', '55741/NV Bán Hàng - CocaCola/Vùng 2', '56500- NV Wiko - Vùng 02', '57833/Nhân viên bán hàng LG', '60824/Spiral-CocaCola/Vùng 1', '64577/NhânViênBánHàng/NNA/Quang Ninh/Vùng1', '70629/NV Bán Hàng - CocaCola', '70977/NhânViênBánHàng/BEKO/Bà Rịa – Vũng Tàu/Vùng1', '72069/Unza/NVBH/Châu Đốc/Vùng 2', '72395/Nhân viên bán hàng VBL/Vùng 2/Cam Le 1', '72771/NV Bán Hàng - CocaCola', '73835/Nhân viên tư vấn bán hàng VBL/Vùng 2/Kiên Giang A1A', '75967/NV Bán Hàng - CocaCola/Vùng 2', '77022Nhân viên bán hàng VBL/Vùng 2/Hai Chau 3', '79341/Nhân viên tư vấn bán hàng VBL/Vùng 2/Kiên Giang B3D', '80198/Nhân viên dự án SRF/Huế/Vùng 3', '81511/Nhân viên dự án SRF/Bình Dương/Vùng 1', '82617/NVKD - CocaCol

In [5]:
def fixmaCv(x):
    lower = str(x).lower()
    if any([x in lower for x in ["nhân viên", "kỹ", "chuyên viên", "kế toán", "cán bộ", "thủ kho", "tạp vụ",\
                                "thu ngân", "may", "cán sự", "phóng viên", "giao dịch", "operator", "cnv",\
                                "huấn luyện viên", "ktoán", "bí thư đoàn", "nv", "lễ tân", "thư ký ",\
                                "gsbh", "tiếp viên", "nhõn viờn", "cb", "chăm sóc", "hướng dẫn viên", \
                                "kiểm phẩm", "chuyờn viờn", "kĩ sư", "kĩ thuật viên", "cấp dưỡng", "kiểm gấp",\
                                "kiểm soát viên", "kiểm ngân", "nv", "lưu hóa", "nhân  viên", "lưu trữ viên", \
                                "phát hành báo", "ktv", "kiến trúc sư", "kinh tế viên", "staff", "checker",\
                                "lập trình viên", "sỹ quan", "dịch vụ", "kiểm tra viên", "chuyªn viªn", \
                                "cộng tác viên", "qc ksqt", "nhân vên", "kế hoạch sản xuất", "nhaõn vieõn", \
                                "thí nghiệm", "thông kê", "kiểm hàng", "k? s?", "thư ký", "hợp đồng", \
                                "thẩm phán", "thao tác viên", "thủ quỹ", "lấy mẫu", "LÁI XE".lower(), \
                                "biên tập", "ksxd", "phiên dịch", "tb tuyên huấn", "sales", "thường trực", \
                                "kĩ thuật", "trợ lý", "tiếp thực", "trưởng công an", "giỏm sỏt", \
                                 "trình dược viên", "btđtn", "phúng viờn", "phường đội", "trực ban", \
                                "thông dịch", "tiếp thị", "tư vấn", "tổng đài", "vi tính", "viên chức", \
                                "văn phòng", "văn thư", 'Thiết Kế'.lower(), "vận hành", "nhan vien", \
                                "kiểm tra chất lượng", "công an viên", "pbt", "phi công", "laptop",\
                                "phụ trách kiểm hàng", "quay phim", "kỷ thuật viên", "thư viện viên", \
                                "trực buồng", "techcombank", "cảng vụ viên", "trainee", "bhxh", "marketing", \
                                "tuyên truyền viên", "an ninh viên", "tổ viên", "kế hoạch", "ks", "công chức", \
                                "an toàn viên", "CHỈ SỐ VIÊN".lower(), "kỹ thuật viên", "quản sinh", \
                                "thu mua", "giám éịnh viên", "kiểm định viên", "bí thư xã  đoàn", "giám định", \
                                "quản trị mạng", "thơợ", "thống kê", "thủ  quỹ", "điều tra viên", \
                                "điện thoại", "điện thọai", "đội thông tin", "đón  tiếp khách hàng", "đăng kiểm",\
                                "đơn hàng", "đại diện", "điều phối viên", "n.v", "bán vé", "c«ng nh©n", \
                                "gs camera", "cỏn bộ", "máy dán", "tv", "đtv", "đồ họa"]]):
        return "NV"
    elif any([x in lower for x in ["tổ trưởng", "giám sát", "quản lý", "cửa hàng trưởng", "chuyền trưởng", "kho",\
                                  "phó", "pp", "nhóm trưởng", "trưởng phòng", "trưởng ca", "trưởng tàu khách", \
                                  "trưởng nhóm", "cửa hàng trưởng", "trưởng bộ", "tổng phụ trách", "đào tạo", \
                                  "phụ trách phòng", "quản doanh", "vhtt", "trưởng ban", "dự án", "nhân sự",\
                                  "chỉ huy", "triển khai", "trưởng kiểm soát", "điều hành", "đội trưởng", \
                                  "chỉ huy", "qlsx", "t? tr??ng"]]):
        return "QL"
    elif any([x in lower for x in ["công", "lái xe", "bảo vệ", "thợ", "lao động", "tài xế", "bán hàng",\
                                "phụ kho", "phụ việc", "phết", "bảo mẫu", "phục vụ", "đóng gói", "ép cao tần", \
                                "cn", "sản Phẩm", "giao nhận","cụng nhõn", "gia công", "kiÓm tra".lower(), \
                                "cô nuôi", "bảo dưỡng", "c.n", "coi bồn", "cạo mủ", "coõng nhaõn", "khai thác",\
                                "cod", "bốc xếp", "bếp", "lái máy", "lđpt", "khuôn", "hàng rong", "phụ bếp",\
                                "phụ tá", "thủ công", "phụ xe", "dệt", "sửa chữa", "băng chuyền", "lắp ráp", \
                                "chà nhám", "cn", "bếp phụ", "chụp hình", "KHUNG SắT".lower(), "kcs chuyền", \
                                "lao động", "thành hình", "lao công", "cônh nhân", "thi công", "cán lần 1", \
                                "lái phụ máy", "thủy thủ", "gác ghi", "quay ghi", "bảo trì", "bảo mẩu", \
                                "máy 3 tàu", "giao hàng", "trực chính", "hỗ trợ", "lái xúc gạt", "công  nhân",\
                                "lđhđ", "lao d?ng", "c. nhân", "pv nước", "hđlđ", "vệ sinh", "phụ", "vệ sỹ", \
                                "xếp hộp", "barista", "hàn điện", "thuỷ thủ", "xây dựng", "b?o v?", \
                                "mài phá", "lái cẩu", "xếp da", "worker", "vân hành", "cong nhan", "pha chế", \
                                "máy trưởng", "phụ trách sản xuất", "trải vải", "YếN TIệC".lower(), "gấp xếp", \
                                "in sơ đồ", "lđ phổ thông", "mài", "nhồi mút", "ép", "gói", "đóng", \
                                "đội viên", "đánh máy", "đứng máy", "ủi", "săm", 'atlđ']]):
        return "CN"
    elif any([x in lower for x in ["giáo viên", "gv", "giảng viên", "chủ nhiệm lớp", "giáo viến", \
                                   "Giáo viến".lower(), "mầm non"]]):
        return "GV"
    elif any([x in lower for x in ["kinh doanh", "kinh  doanh"]]):
        return "KD"

    elif any([x in lower for x in ["hiệu trưởng", "chủ tịch", "pgđ", "giám đốc"]]):
        return "GĐ"
    elif any([x in lower for x in ["điều dưỡng", "hộ lý", "nữ hộ sinh", "h? lý", "hộ sinh"]]):
        return "ĐD"
    elif any([x in lower for x in ["y sĩ", "dược sỹ", "bác sĩ", "bác sỹ", "dược", "dược tá", "thú y", "y sỹ",\
                                  "y tế", "y công", "y tá", "dựơc tá", "ytth"]]):
        return "BS"
    elif any([x in lower for x in ["cử nhân", "cđ", "học viên", "tc", "cao đẳng", "cao đẵng", "trung cấp"]]):
        return "HS"
    elif any([x in lower for x in ["diễn viên", "ca sĩ"]]):
        return "DV-CS"
    elif any([x in lower for x in ["undefined", "nguyễn thế hưng", "ln091", "thieu chuc danh", "th noọng hẹt", \
                                  "875 lk", "-", "p.xnk", "cỏn b?", '22568', "unt", "01019882", "xtnt", "ltv", \
                                  "plđg hc", "mia", "pqtd", "kttntl", "llqs", "kcsmhm", "nt", "dtm", "i", "qc",\
                                  '/  /', '1', '14', 'a', 'chủ hộ', 'coõng vuù', 'kcs', 'pk b2c',\
                                   'xt2', 'kcs', 'sxct']]):
        return np.nan
    return x

In [6]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [7]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_45", "Field_48", "Field_49", "Field_56", 'homeTownName', "currentLocationCity", \
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["Field_18", "Field_68", "Field_46"]
}

In [8]:
lst = data_type["cat_num"] + data_type["cat_str"] + data_type["date"] + data_type["address"] + data_type["drop"]

In [9]:
"""
for f in df_transform.columns:
    vc = df_transform[f].value_counts()
    print(f"{f} : {len(vc)} : {vc.index[:10]}")
"""

'\nfor f in df_transform.columns:\n    vc = df_transform[f].value_counts()\n    print(f"{f} : {len(vc)} : {vc.index[:10]}")\n'

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [10]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [11]:
def fix38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    if x == 1.0 or x == "1":
        return 1
    return np.nan

In [12]:
def fix54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    if x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [13]:
def fix55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [14]:
def fix61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [15]:
def fixcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [16]:
def fixcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [17]:
def fixhomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [18]:
def fixhomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [19]:
def fixdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [20]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [21]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df

In [22]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [23]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [24]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col]
        
    return df

In [25]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize)
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize)

    return df

In [26]:
df_transform["maCv"]

0         Nhân viên bán hàng trực tiếp
1                                  NaN
2                                  NaN
3                                  NaN
4                                  NaN
                     ...              
20376                              NaN
20377                              NaN
20378                              NaN
20379                              NaN
20380    Nhân viên chăm sóc khách hàng
Name: maCv, Length: 73411, dtype: object

In [27]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fix38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fix54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fix55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fix61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fixcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fixcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fixhomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fixhomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fixdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)
df_transform["maCv"] = df_transform["maCv"].apply(fixmaCv)
df_transform["null_sum"] = df_transform.isnull().sum(axis=1)

df_transform = process_gender(df_transform)
df_transform = process_datetime_cols(df_transform)

df_transform = process_location(df_transform)

In [28]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [29]:
categoricals = []
for header in df_transform.columns:
    if str(df_transform[header].dtypes) is "object":
        categoricals.append(header)
for col in categoricals:
    df_transform[col] = df_transform[col].astype(str)
lbl = LabelEncoder()
for col in categoricals:
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    df_transform[col] = df_transform[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(df_transform[col]))
    
    # Take if vals appeared both 5 times
    common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 100].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    df_transform.loc[~df_transform[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(df_transform[col].tolist())
    df_transform[col] = lbl.transform(df_transform[col])


In [30]:
fill_1 = []
for header in df_transform.columns:
    vc = df_transform[header].value_counts()
    if len(vc) < 2:
        fill_1.append(header)
for col in fill_1:
    df_transform[col] = df_transform[col].fillna(-1)

In [31]:
df_transform = df_transform.fillna(df_transform.mean())

In [32]:
import re
df_transform = df_transform.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [34]:
train_dev = pd.concat([df_train.id, df_train.label, df_transform.iloc[:len(df_train),:]], axis = 1)
test = pd.concat([df_test.id, df_transform.iloc[len(df_train):,:]], axis = 1)

In [37]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'min_data_in_leaf': 200,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'min_hessian': 0.01,
    'feature_fraction_seed': 2,
    'bagging_seed': 3
}

In [38]:
def compute_gini(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    gini = 2 * auc - 1
    return gini

In [39]:
iter_predicttion = []
for i in range(1):
    pred_stack = []
    kf = KFold(n_splits = 4, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,2:]
        dev1 = train_dev.iloc[dev_index,2:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test.iloc[:,1:])
        predictions_dev = clf.predict(dev1)
        pred_stack.append(predictions_test)
        print(compute_gini(train_dev.loc[dev_index]["label"], predictions_dev))
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

0.4822043663327342
0.480482848745021
0.46271037410936877
0.47580560425143203


In [40]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

In [41]:
data_version = "10-10-2020-d"

In [42]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)